In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX_Books.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX-Book-Ratings.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX-Users.csv


In [2]:
df = pd.read_csv('/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv')
df.drop(['Unnamed: 0','img_s','img_m','img_l'], axis=1, inplace=True)
df.head()

,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [3]:
df.drop_duplicates(subset=['user_id','book_title'],inplace=True)
df.dropna(inplace=True)
df.shape

(978141, 15)

In [4]:
counts = df['user_id'].value_counts()
keep = counts[counts > 10].index
keep

Int64Index([198711,  98391, 153662,  35859, 212898, 278418,  76352, 110973,
            235105,  16795,
            ...
            240033,  91685,  26030,  51511, 148121, 221967, 244892, 223375,
            238527, 246268],
           dtype='int64', length=10234)

In [5]:
df = df[df['user_id'].isin(keep)]
df.shape

(817015, 15)

# Start with a user-based and item-based collab filtering model

In [6]:
counts = df['book_title'].value_counts()
keep = counts[counts > 10].index
keep

Index(['Wild Animus', 'The Lovely Bones: A Novel', 'The Da Vinci Code',
       'The Nanny Diaries: A Novel', 'Bridget Jones's Diary',
       'The Secret Life of Bees',
       'Divine Secrets of the Ya-Ya Sisterhood: A Novel', 'A Painted House',
       'Angels & Demons', 'Life of Pi',
       ...
       'Losing Gemma', 'Girl With Curious Hair (Norton Paperback Fiction)',
       'Track of the Cat', 'Tangled Up in Blue', 'The Very Thought of You',
       'Romancing the Rogue (A Kenyon Family Novel)',
       'Letourneau's Used Auto Parts', 'Der Rabbi',
       'Der Richter Und Sein Henker',
       'Little House on the Prairie (Little House)'],
      dtype='object', length=12337)

In [7]:
df = df[df['book_title'].isin(keep)]
df.shape

(398552, 15)

In [8]:
pivot_df = df.pivot(index='user_id',columns='book_title',values='rating')

In [9]:
pivot_df

book_title,"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,01-01-00: The Novel of the Millennium,"1,000 Places to See Before You Die",10 Lb. Penalty,1001 Ways to Be Romantic,101 Dalmatians,101 Great American Poems (Dover Thrift Editions),...,Zombies of the Gene Pool,Zoya,Zoya's Story: An Afghan Woman's Struggle for Freedom,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\A\"" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,one hundred years of solitude,stardust
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
avg_ratings = pivot_df.mean(axis=1)
user_rating_df = pivot_df.sub(avg_ratings,axis=0).fillna(0)
user_rating_df

book_title,"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,01-01-00: The Novel of the Millennium,"1,000 Places to See Before You Die",10 Lb. Penalty,1001 Ways to Be Romantic,101 Dalmatians,101 Great American Poems (Dover Thrift Editions),...,Zombies of the Gene Pool,Zoya,Zoya's Story: An Afghan Woman's Struggle for Freedom,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\A\"" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,one hundred years of solitude,stardust
user_id,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.623529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
from sklearn.neighbors import KNeighborsRegressor

In [12]:
def predict_rating(df,user,book):
    X = df.drop(book,axis=1)
    y = df[book]
    knn = KNeighborsRegressor(metric='cosine',n_neighbors=20)
    knn.fit(X,y)
    return knn.predict([X.iloc[user]])

In [13]:
predict_rating(user_rating_df,743,'A Painted House')

array([0.])

In [14]:
# data here is wayyyy to spare - we need to apply matrix factorization

In [15]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(user_rating_df)

In [16]:
sigma = np.diag(sigma)
recalc = np.dot(np.dot(U,sigma),Vt)
recalc = recalc + avg_ratings.values.reshape(-1,1)

In [17]:
recalc_df = pd.DataFrame(recalc,index=user_rating_df.index,columns=user_rating_df.columns)
recalc_df

book_title,"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,01-01-00: The Novel of the Millennium,"1,000 Places to See Before You Die",10 Lb. Penalty,1001 Ways to Be Romantic,101 Dalmatians,101 Great American Poems (Dover Thrift Editions),...,Zombies of the Gene Pool,Zoya,Zoya's Story: An Afghan Woman's Struggle for Freedom,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\A\"" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,one hundred years of solitude,stardust
user_id,,,,,,,,,,,,,,,,,,,,,
8,1.666993e+00,1.666958e+00,1.663960e+00,1.666747e+00,1.666875e+00,1.666960e+00,1.663321e+00,1.666276e+00,1.665277e+00,1.666676e+00,...,1.663111e+00,1.667612e+00,1.666646e+00,1.666659e+00,1.665880e+00,1.668249e+00,1.667044e+00,1.664101e+00,1.666820e+00,1.666653e+00
99,5.143376e+00,5.143373e+00,5.139840e+00,5.143994e+00,5.143505e+00,5.143220e+00,5.143671e+00,5.142381e+00,5.147066e+00,5.142586e+00,...,5.142649e+00,5.134708e+00,5.143045e+00,5.141543e+00,5.141558e+00,5.140813e+00,5.143196e+00,5.141447e+00,5.143307e+00,5.142552e+00
242,6.493622e+00,6.495104e+00,6.534044e+00,6.496985e+00,6.494897e+00,6.494884e+00,6.508907e+00,6.507202e+00,6.508258e+00,6.503132e+00,...,6.505868e+00,6.502282e+00,6.498828e+00,6.501862e+00,6.508171e+00,6.508418e+00,6.498301e+00,6.518539e+00,6.494997e+00,6.501163e+00
243,1.728079e+00,1.729003e+00,1.697317e+00,1.726535e+00,1.726279e+00,1.728986e+00,1.709748e+00,1.724835e+00,1.724081e+00,1.725701e+00,...,1.718247e+00,1.725908e+00,1.726924e+00,1.726597e+00,1.719949e+00,1.745502e+00,1.727857e+00,1.706401e+00,1.727163e+00,1.726517e+00
254,1.602226e+00,1.618772e+00,1.587702e+00,1.602192e+00,1.591995e+00,1.608234e+00,1.602088e+00,1.668773e+00,1.637696e+00,1.646346e+00,...,1.624787e+00,1.645847e+00,1.618946e+00,1.634659e+00,1.634133e+00,1.722596e+00,1.623732e+00,1.660361e+00,1.600119e+00,1.642703e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278633,4.176194e+00,4.174988e+00,4.230828e+00,4.181692e+00,4.179408e+00,4.174600e+00,4.224034e+00,4.182843e+00,4.202476e+00,4.179874e+00,...,4.210412e+00,4.153664e+00,4.179732e+00,4.175684e+00,4.190781e+00,4.142137e+00,4.176424e+00,4.219428e+00,4.178454e+00,4.178439e+00
278637,-3.888734e-18,-4.296563e-18,4.096927e-17,5.536300e-18,6.897609e-19,-9.321692e-19,-1.013166e-17,-2.285609e-18,2.870732e-17,-1.399032e-18,...,-1.565692e-17,-3.179891e-17,1.040475e-18,-2.775557e-18,2.949073e-19,3.069936e-17,1.624469e-18,-1.689336e-17,-1.387617e-18,-4.906681e-18
278771,-5.488366e-18,-6.067689e-18,5.718506e-17,7.065044e-18,6.664167e-19,-1.333188e-18,-1.361502e-17,-3.024530e-18,3.740735e-17,-1.923820e-18,...,-2.004684e-17,-3.961232e-17,1.439974e-18,-3.111546e-18,1.050806e-18,4.314425e-17,1.987548e-18,-2.258727e-17,-1.996906e-18,-6.614348e-18


In [18]:
def predict_rating(df,user,book):
    X = df.drop(book,axis=1)
    y = df[book]
    knn = KNeighborsRegressor(metric='cosine',n_neighbors=20)
    knn.fit(X,y)
    return knn.predict([X.iloc[user]])

In [19]:
predict_rating(recalc_df,743,'A Painted House')

array([3.48975246])

# Hybrid Model